In [1]:
import numpy as np
import pandas as pd
import random
random.seed(123)

In [24]:
class Sampler:
    def __init__(self, A, B, M, N, prime, dist, iteration):
        self.A = A # number of hash fn
        self.B = B # number of counters
        self.dist = dist # which distribution to use
        self.M = M # for range of number generated 
        self.N = N # length of stream
        self.p = prime # prime number for the hash function
        self.iteration = iteration
        self.a = np.random.choice(range(1, self.p-1), A) # list of a for hash fn
        self.b = np.random.choice(range(0, self.p-1), A) # list of b for hash fn
        
    # generate uniform distributed sample
    def unif_dist(self):
        return(np.floor(np.random.uniform(0, self.M, self.N)))

    # generate exp distributed sample
    def exp_dist(self):
        return(np.floor(np.random.exponential(scale=(1/-np.log(0.5)), size = self.N)))
        
    # prime field hash function
    def hashfn(self, x, a, b):      # improve by adding tests of p and B
        return(((a*x + b)%self.p)%(self.B))
    
    # query 1
    def query1(self, x, C):
        keep_c = []
        for i in enumerate(range(self.A)):
            keep_c.append(C[i[0], self.hashfn(x, self.a[i[0]], self.b[i[0]])])
        return(np.median(keep_c))

    # query 2
    def query2(self, x, C):
        keep_c = []
        for i in enumerate(range(self.A)):  # to improve, catch the exception when B is not even number...
            coln = self.hashfn(x, self.a[i[0]], self.b[i[0]])
            if coln % 2 == 0:
                est = C[i[0], coln] - C[i[0], coln + 1]
                keep_c.append(est)
            elif coln % 2 == 1:
                est = C[i[0], coln] - C[i[0], coln - 1]
                keep_c.append(est)            
        return(np.median(keep_c))

    # populate counter matrix with only n/N of the data
    def populate(self):
        C = np.zeros((self.A, self.B)) # initiatize counter matrix
        if self.dist == 'unif':
            sample = self.unif_dist()
        elif self.dist == 'exp':
            sample = self.exp_dist()
        for i in enumerate(sample):
            for j in enumerate(range(self.A)):
                    C[j[0], self.hashfn(i[1], self.a[j[0]], self.b[j[0]]).astype(int)] += 1
        return(C)
                    
    def result(self):
        if self.dist =='unif':
            expected_dis = np.array(self.N/self.M)
        elif self.dist == 'exp':
            expected_dis = []
            for i in enumerate(range(self.M)):
                expected_dis.append(0.5**(i[0]+1) * self.N)
    
        expected_dis = np.array(expected_dis)
        keep_mae_query1 = []
        keep_mae_query2 = []
        for i in enumerate(range(self.iteration)):
            C = self.populate()
            out1 = []
            out2 = []
            for j in enumerate(range(self.M)):
                query1_ret = self.query1(j[0], C)
                out1.append(query1_ret)
                query2_ret = self.query2(j[0], C)
                out2.append(query2_ret)
                
            keep_mae_query1.append(np.mean(np.abs(out1 - expected_dis)))
            keep_mae_query2.append(np.mean(np.abs(out2 - expected_dis)))
        
        return(np.round((np.mean(keep_mae_query1), np.mean(keep_mae_query2)),2))
        

In [15]:
a1 = Sampler(A=20, B=20, M=10, N=2000, prime= 41, dist='exp', iteration=1)
answer = a1.result()
print('A=',a, ', B=',b,', M=',m,', N=',n,', Dist=Exp, Query1 out = ', answer[0], 'Query2 out = ', answer[1]) ## 2000

(7.388843749999999, 7.38534375)

In [33]:
# test 1
a = 10
b = 10
M_list = [10, 15, 25]
N_list = [1000, 2000, 5000]

# comparing which algorithm works better for which distribution
for m in M_list:
    for n in N_list:
        a1 = Sampler(A=a, B=b, M=m, N=n, prime= 41, dist='exp', iteration=100)
        answer = a1.result()
        print('A=',a, ', B=',b,', M=',m,', N=',n,', Dist=Exp, Query1 out = ', answer[0], ', Query2 out = ', answer[1])
        a2 = Sampler(A=a, B=b, M=m, N=n, prime= 41, dist='unif', iteration=100)
        answer = a2.result()
        print('A=',a, ', B=',b,', M=',m,', N=',n,', Dist=Uni, Query1 out = ', answer[0], ', Query2 out = ', answer[1]) ## 2000

A= 10 , B= 10 , M= 10 , N= 1000 , Dist=Exp, Query1 out =  5.82 , Query2 out =  26.7
A= 10 , B= 10 , M= 10 , N= 1000 , Dist=Unif, Query1 out =  32.4 , Query2 out =  68.23
A= 10 , B= 10 , M= 10 , N= 2000 , Dist=Exp, Query1 out =  67.03 , Query2 out =  28.16
A= 10 , B= 10 , M= 10 , N= 2000 , Dist=Unif, Query1 out =  107.97 , Query2 out =  85.24
A= 10 , B= 10 , M= 10 , N= 5000 , Dist=Exp, Query1 out =  30.41 , Query2 out =  66.06
A= 10 , B= 10 , M= 10 , N= 5000 , Dist=Unif, Query1 out =  105.82 , Query2 out =  279.08
A= 10 , B= 10 , M= 15 , N= 1000 , Dist=Exp, Query1 out =  8.88 , Query2 out =  14.91
A= 10 , B= 10 , M= 15 , N= 1000 , Dist=Unif, Query1 out =  62.59 , Query2 out =  44.06
A= 10 , B= 10 , M= 15 , N= 2000 , Dist=Exp, Query1 out =  18.68 , Query2 out =  28.41
A= 10 , B= 10 , M= 15 , N= 2000 , Dist=Unif, Query1 out =  131.99 , Query2 out =  52.94
A= 10 , B= 10 , M= 15 , N= 5000 , Dist=Exp, Query1 out =  42.06 , Query2 out =  63.88
A= 10 , B= 10 , M= 15 , N= 5000 , Dist=Unif, Quer

In [35]:
n = 10000
a1 = Sampler(A=a, B=b, M=m, N=n, prime= 41, dist='exp', iteration=100)
answer = a1.result()
print('A=',a, ', B=',b,', M=',m,', N=',n,', Dist=Exp, Query1 out = ', answer[0], ', Query2 out = ', answer[1])
a2 = Sampler(A=a, B=b, M=m, N=n, prime= 41, dist='unif', iteration=100)
answer = a2.result()
print('A=',a, ', B=',b,', M=',m,', N=',n,', Dist=Uni, Query1 out = ', answer[0], ', Query2 out = ', answer[1]) ## 2000

A= 10 , B= 10 , M= 25 , N= 10000 , Dist=Exp, Query1 out =  251.16 , Query2 out =  204.77
A= 10 , B= 10 , M= 25 , N= 10000 , Dist=Uni, Query1 out =  711.82 , Query2 out =  290.13


In [36]:
n = 10000
m = 100
a1 = Sampler(A=a, B=b, M=m, N=n, prime= 41, dist='exp', iteration=100)
answer = a1.result()
print('A=',a, ', B=',b,', M=',m,', N=',n,', Dist=Exp, Query1 out = ', answer[0], ', Query2 out = ', answer[1])
a2 = Sampler(A=a, B=b, M=m, N=n, prime= 41, dist='unif', iteration=100)
answer = a2.result()
print('A=',a, ', B=',b,', M=',m,', N=',n,', Dist=Uni, Query1 out = ', answer[0], ', Query2 out = ', answer[1]) ## 2000

A= 10 , B= 10 , M= 100 , N= 10000 , Dist=Exp, Query1 out =  370.06 , Query2 out =  490.16
A= 10 , B= 10 , M= 100 , N= 10000 , Dist=Uni, Query1 out =  894.72 , Query2 out =  95.03


In [60]:
# test 2 
A_list = [10, 15, 25]
B_list = [14, 18, 30]
m = 40
n = 5000

# comparing which algorithm works better for which distribution
for a in A_list:
    for b in B_list:
        a1 = Sampler(A=a, B=b, M=m, N=n, prime= 41, dist='exp', iteration=100)
        answer = a1.result()
        print('A=',a, ', B=',b,', M=',m,', N=',n,', Dist=Exp, Query1 out = ', answer[0], ', Query2 out = ', answer[1])
        a2 = Sampler(A=a, B=b, M=m, N=n, prime= 41, dist='unif', iteration=100)
        answer = a2.result()
        print('A=',a, ', B=',b,', M=',m,', N=',n,', Dist=Uni, Query1 out = ', answer[0], ', Query2 out = ', answer[1]) ## 2000

A= 10 , B= 14 , M= 40 , N= 5000 , Dist=Exp, Query1 out =  20.53 , Query2 out =  89.11
A= 10 , B= 14 , M= 40 , N= 5000 , Dist=Uni, Query1 out =  247.5 , Query2 out =  121.83
A= 10 , B= 18 , M= 40 , N= 5000 , Dist=Exp, Query1 out =  21.39 , Query2 out =  38.44
A= 10 , B= 18 , M= 40 , N= 5000 , Dist=Uni, Query1 out =  153.6 , Query2 out =  122.92
A= 10 , B= 30 , M= 40 , N= 5000 , Dist=Exp, Query1 out =  3.08 , Query2 out =  12.3
A= 10 , B= 30 , M= 40 , N= 5000 , Dist=Uni, Query1 out =  64.7 , Query2 out =  123.85
A= 15 , B= 14 , M= 40 , N= 5000 , Dist=Exp, Query1 out =  6.54 , Query2 out =  12.68
A= 15 , B= 14 , M= 40 , N= 5000 , Dist=Uni, Query1 out =  247.31 , Query2 out =  122.85
A= 15 , B= 18 , M= 40 , N= 5000 , Dist=Exp, Query1 out =  4.02 , Query2 out =  12.01
A= 15 , B= 18 , M= 40 , N= 5000 , Dist=Uni, Query1 out =  145.07 , Query2 out =  123.57
A= 15 , B= 30 , M= 40 , N= 5000 , Dist=Exp, Query1 out =  2.98 , Query2 out =  3.27
A= 15 , B= 30 , M= 40 , N= 5000 , Dist=Uni, Query1 out